<a href="https://colab.research.google.com/github/dcolinmorgan/netZooPy/blob/gpu-panda/gpuPanda_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0.5: Change colab runtime type to "GPU"

# Accelerated gene regulation estimation using gpuPANDA

Author : Daniel Morgan

#1. Setup Environment: download, install & import latest packages

In [0]:
import os
import s3fs
import pandas as pd
import numpy as np
from psutil import *

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


###  
##1.1 Configure cupy for installed Cuda version
1.   use cuda version based on above release (*e.g.* release 10.1= cuda101)
2.   install cupy version 7.4.0 or above 



In [12]:
!pip install cupy-cuda101==7.4.0
import cupy as cp
# print(cp.__version__)
!pip freeze | grep cupy

cupy-cuda101==7.4.0


In [0]:
!git clone --single-branch --branch gpu-panda https://github.com/dcolinmorgan/netZooPy.git
os.chdir('netZooPy')
!pip install -e .
os.chdir('..')

In [0]:
import netZooPy
from netZooPy.panda import Panda

#2. Load test data

In [0]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/cells/ppi/LCL_ppi.txt',sep='\t',header=None)
LCL_expression=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/expression/Hugo_exp1_lcl.txt',sep='\t',header=None)
# LCL_motif=pd.read_csv('../Hugo_motifCellLine_reduced.txt')
LCL_ppi.to_csv('LCL_ppi.txt',sep='\t',index=False)
LCL_expression.to_csv('Hugo_exp1_lcl.txt',sep='\t',index=False)


In [0]:
# os.getcwd()
expression_data='Hugo_exp1_lcl.txt'
motif_data='Hugo_motifCellLine_reduced.txt'
ppi_data='LCL_ppi.txt'


#3. Run Panda with GPU flag as 4th arguement

In [23]:
np.set_printoptions(precision=17)
panda_obj = Panda(expression_data, motif_data, ppi_data,computing='gpu',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')

Loading motif data ...
  Elapsed time: 0.30 sec.
Loading expression data ...
Expression matrix: (27150, 127)
  Elapsed time: 0.66 sec.
Loading PPI data ...
Number of PPIs: 85315
  Elapsed time: 0.03 sec.
(1000, 127)
(27150, 127)
Calculating coexpression network ...
  Elapsed time: 0.01 sec.
Creating motif network ...
  Elapsed time: 0.42 sec.
Creating PPI network ...
  Elapsed time: 0.06 sec.
Normalizing networks ...
  Elapsed time: 0.05 sec.
Clearing motif and ppi data, unique tfs, and gene names for speed
Running GPU PANDA algorithm ...
step: 0, hamming: 0.8135873745606481
step: 1, hamming: 0.28784066641507783
step: 2, hamming: 0.18948187205166064
step: 3, hamming: 0.14761716864546812
step: 4, hamming: 0.12336583880810298
step: 5, hamming: 0.10634375670636508
step: 6, hamming: 0.09285137142779426
step: 7, hamming: 0.0813786407467249
step: 8, hamming: 0.07126892739697534
step: 9, hamming: 0.062227189395228276
step: 10, hamming: 0.05411012736489582
step: 11, hamming: 0.0468409544097808

In [24]:
panda_obj.panda_network[1,1]

array(-0.46963516199079375)

In [25]:
panda_obj.panda_network.shape

(652, 1000)

In [27]:
!nvidia-smi

Mon Apr 27 02:43:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    36W / 250W |    381MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [28]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.158
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multi